In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
import pickle as pkl

In [ ]:
## Load the catalog
with open('pkl/cat_flambda_photoz_gaia.pkl', 'rb') as file:
    cat = pkl.load(file)

In [ ]:
tcurves = load_tcurves(load_filter_tags())

In [ ]:
# Define the array of the filters we are going to use // gSDSS band
nb_ind_arr = [8, 9, 10, 11, 12, 13, 14, 15]
bb_ind = -3

In [ ]:
# Function to drop sources
def make_masks(cat, nb_ind):
    # Mask sources with high photoz odds
    mask_pz_odds = cat['odds'] < 0.98
    # Mask sources with proper motion according to Gaia (sigma > 3)
    parallax_sn = np.abs(cat['parallax'] / cat['parallax_err'])
    pmra_sn = np.abs(cat['pmra'] / cat['pmra_err'])
    pmdec_sn = np.abs(cat['pmdec'] / cat['pmdec_err'])
    mask_pmotion = (
        (np.sqrt(parallax_sn**2 + pmra_sn**2 + pmdec_sn**2) < 27.**0.5)
        | ( np.isnan(parallax_sn) | np.isnan(pmra_sn) | np.isnan(pmdec_sn) )
    )
    # Mask sources with SNR < 5 in the selected NB
    mask_snr = cat['flx_err'][:,nb_ind] < 0.2
    
    mask_total = mask_pz_odds & mask_pmotion & mask_snr
    return mask_total

In [ ]:
# The model function
def model_f(x, m, b):
    return m*x + b

In [ ]:
nb_ind = 11
mask = make_masks(cat, nb_ind)
pm_flx = cat['flx'][mask] * 1e-19
pm_err = cat['flx_err'][mask] * pm_flx
filters_tags = load_filter_tags()
fwhm_nb = nb_fwhm(tcurves, nb_ind, True)
nb_fwhm_Arr = [nb_fwhm(tcurves, idx, True) for idx in np.arange(len(filters_tags))]
w_central = np.array(central_wavelength(load_tcurves(load_filter_tags())))
N_nb = 5 # Number of nb on each side of the central one
ew0min = 50
cont_stack, cont_err_stack = stack_estimation(pm_flx.T, pm_err.T, nb_ind, N_nb, w_central, nb_fwhm_Arr, ew0min)
_, cf, cont_err = nbex_cont_estimate(model_f, pm_flx, pm_err, nb_ind, w_central, N_nb, ew0min, fwhm_nb)

In [ ]:
z = 1215.67 / w_central[nb_ind] - 1
ew = ew0min*(1 + z)
line = pm_flx[:, nb_ind] - cont_stack - (ew * cont_stack) / fwhm_nb\
            > 3 * (cont_err_stack**2 + pm_err[:, nb_ind]**2)**0.5

len(np.where(line)[0])

In [ ]:
bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
j = 0
for i in range(len(line)):
    if ~line[i]: continue
    j += 1
    if j > 20: break
        
    pm = pm_flx[i]
    errors = pm_err[i]
    cont_fit = cf[i]
    fit_err = cont_err[i]
    
    fig, ax = plt.subplots(figsize=(12,9))
    ax.scatter(w_central[:-3], pm[:-3])
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], fmt='none')
    ax.scatter(w_central[nb_ind], pm[nb_ind], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4], xerr=bb_fwhm[-4]/2, yerr = errors[-4],
                fmt='none', color='purple', elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], xerr=bb_fwhm[-3]/2, yerr = errors[-3],
                fmt='none', color='green', elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], xerr=bb_fwhm[-2]/2, yerr = errors[-2],
                fmt='none', color='red', elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], xerr=bb_fwhm[-1]/2, yerr = errors[-1],
                fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    # Fit line
    w_fit_min = w_central[nb_ind - N_nb]
    w_fit_max = w_central[nb_ind + N_nb]
    x_fit = np.linspace(w_fit_min, w_fit_max, 1000)
    y_fit = cont_fit[1] + cont_fit[0]*x_fit
    ax.errorbar(w_central[nb_ind]+10, cont_stack[i], yerr=cont_err_stack[i],
                c='darkorchid', marker='^', markersize=9,
                capsize=4, label='Stack NBs', elinewidth=2, capthick=2)
    cont_fit_value = cont_fit[1] + cont_fit[0]*w_central[nb_ind]
    ax.plot(x_fit, y_fit, c='mediumseagreen', linestyle='dashed')
    ax.errorbar(w_central[nb_ind], cont_fit_value, yerr=cont_err[i],
                c='mediumseagreen', marker='*', markersize=9,
                capsize=4, label='Linear fit', elinewidth=2, capthick=2)
    
    plt.legend()
    
    print(cat['number'][mask][i])
    
    plt.show()